In [1]:
import sqlite3
import json
from datetime import datetime
import time

In [2]:
timeframe = '2015-01'
sql_transaction = []
start_row = 0
cleanup = 1000000

In [3]:
connection = sqlite3.connect('{}.db'.format(timeframe))
c = connection.cursor()

In [4]:
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

In [5]:
def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

In [6]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []


In [7]:
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [8]:
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [9]:
def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [10]:
def acceptable(data):
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    elif len(data) > 32000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

In [11]:
def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [12]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        #print(str(e))
        return False

In [13]:
if __name__ == '__main__':
    create_table()
    row_counter = 0
    paired_rows = 0

    #with open('J:/chatdata/reddit_data/{}/RC_{}'.format(timeframe.split('-')[0],timeframe), buffering=1000) as f:
    with open('D:\DEAN\CHATBOTS\RC_2015-01'.format(timeframe), buffering=1000) as f:
        for row in f:
            #print(row)
            #time.sleep(555)
            row_counter += 1

            if row_counter > start_row:
                try:
                    row = json.loads(row)
                    parent_id = row['parent_id'].split('_')[1]
                    body = format_data(row['body'])
                    created_utc = row['created_utc']
                    score = row['score']
                    
                    comment_id = row['id']
                    
                    subreddit = row['subreddit']
                    parent_data = find_parent(parent_id)
                    
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            if acceptable(body):
                                sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                
                    else:
                        if acceptable(body):
                            if parent_data:
                                if score >= 2:
                                    sql_insert_has_parent(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                    paired_rows += 1
                            else:
                                sql_insert_no_parent(comment_id,parent_id,body,subreddit,created_utc,score)
                except Exception as e:
                    print(str(e))
                            
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

            if row_counter > start_row:
                if row_counter % cleanup == 0:
                    print("Cleanin up!")
                    sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                    c.execute(sql)
                    connection.commit()
                    c.execute("VACUUM")
                    connection.commit()

Total Rows Read: 100000, Paired Rows: 6105, Time: 2018-01-17 13:36:05.675456
Total Rows Read: 200000, Paired Rows: 13837, Time: 2018-01-17 13:36:33.782940
Total Rows Read: 300000, Paired Rows: 22238, Time: 2018-01-17 13:37:02.740443
Total Rows Read: 400000, Paired Rows: 30681, Time: 2018-01-17 13:37:30.840582
Total Rows Read: 500000, Paired Rows: 38915, Time: 2018-01-17 13:37:59.507318
Total Rows Read: 600000, Paired Rows: 46399, Time: 2018-01-17 13:38:27.620751
Total Rows Read: 700000, Paired Rows: 54145, Time: 2018-01-17 13:38:55.969912
Total Rows Read: 800000, Paired Rows: 62623, Time: 2018-01-17 13:39:25.344709
Total Rows Read: 900000, Paired Rows: 71345, Time: 2018-01-17 13:39:53.769487
Total Rows Read: 1000000, Paired Rows: 79923, Time: 2018-01-17 13:40:21.310884
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 87076, Time: 2018-01-17 13:40:54.986833
Total Rows Read: 1200000, Paired Rows: 95201, Time: 2018-01-17 13:41:22.871705
Total Rows Read: 1300000, Paired Rows: 103703, Tim

Total Rows Read: 10300000, Paired Rows: 881670, Time: 2018-01-17 14:26:29.723956
Total Rows Read: 10400000, Paired Rows: 890341, Time: 2018-01-17 14:26:59.554022
Total Rows Read: 10500000, Paired Rows: 898732, Time: 2018-01-17 14:27:29.542047
Total Rows Read: 10600000, Paired Rows: 906661, Time: 2018-01-17 14:27:59.650136
Total Rows Read: 10700000, Paired Rows: 914864, Time: 2018-01-17 14:28:27.261199
Total Rows Read: 10800000, Paired Rows: 923680, Time: 2018-01-17 14:28:55.667190
Total Rows Read: 10900000, Paired Rows: 932871, Time: 2018-01-17 14:29:24.024389
Total Rows Read: 11000000, Paired Rows: 942661, Time: 2018-01-17 14:29:52.581975
Cleanin up!
Total Rows Read: 11100000, Paired Rows: 950508, Time: 2018-01-17 14:30:39.237295
Total Rows Read: 11200000, Paired Rows: 959841, Time: 2018-01-17 14:31:07.767251
Total Rows Read: 11300000, Paired Rows: 969363, Time: 2018-01-17 14:31:36.881649
Total Rows Read: 11400000, Paired Rows: 979253, Time: 2018-01-17 14:32:05.989702
Total Rows Read:

Total Rows Read: 20200000, Paired Rows: 1750493, Time: 2018-01-17 15:18:13.642261
Total Rows Read: 20300000, Paired Rows: 1759576, Time: 2018-01-17 15:18:45.007510
Total Rows Read: 20400000, Paired Rows: 1768972, Time: 2018-01-17 15:19:15.485518
Total Rows Read: 20500000, Paired Rows: 1777355, Time: 2018-01-17 15:19:45.948304
Total Rows Read: 20600000, Paired Rows: 1785345, Time: 2018-01-17 15:20:15.364474
Total Rows Read: 20700000, Paired Rows: 1793385, Time: 2018-01-17 15:20:44.799044
Total Rows Read: 20800000, Paired Rows: 1801389, Time: 2018-01-17 15:21:12.678330
Total Rows Read: 20900000, Paired Rows: 1809957, Time: 2018-01-17 15:21:41.390776
Total Rows Read: 21000000, Paired Rows: 1818270, Time: 2018-01-17 15:22:11.130434
Cleanin up!
Total Rows Read: 21100000, Paired Rows: 1824409, Time: 2018-01-17 15:23:11.010786
Total Rows Read: 21200000, Paired Rows: 1831933, Time: 2018-01-17 15:23:41.980638
Total Rows Read: 21300000, Paired Rows: 1840401, Time: 2018-01-17 15:24:11.700413
Tota

Total Rows Read: 30100000, Paired Rows: 2612803, Time: 2018-01-17 16:13:33.655022
Total Rows Read: 30200000, Paired Rows: 2621430, Time: 2018-01-17 16:14:03.597034
Total Rows Read: 30300000, Paired Rows: 2630552, Time: 2018-01-17 16:14:33.493662
Total Rows Read: 30400000, Paired Rows: 2638650, Time: 2018-01-17 16:15:02.386530
Total Rows Read: 30500000, Paired Rows: 2646280, Time: 2018-01-17 16:15:29.781881
Total Rows Read: 30600000, Paired Rows: 2654124, Time: 2018-01-17 16:15:57.810794
Total Rows Read: 30700000, Paired Rows: 2660741, Time: 2018-01-17 16:16:26.571750
Total Rows Read: 30800000, Paired Rows: 2668925, Time: 2018-01-17 16:16:57.950362
Total Rows Read: 30900000, Paired Rows: 2677493, Time: 2018-01-17 16:17:29.312314
Total Rows Read: 31000000, Paired Rows: 2685640, Time: 2018-01-17 16:17:58.729805
Cleanin up!
Total Rows Read: 31100000, Paired Rows: 2693005, Time: 2018-01-17 16:19:11.412929
Total Rows Read: 31200000, Paired Rows: 2701603, Time: 2018-01-17 16:19:41.816273
Tota

Total Rows Read: 40000000, Paired Rows: 3473242, Time: 2018-01-17 17:09:59.109872
Cleanin up!
Total Rows Read: 40100000, Paired Rows: 3480214, Time: 2018-01-17 17:11:23.158899
Total Rows Read: 40200000, Paired Rows: 3488289, Time: 2018-01-17 17:11:52.515765
Total Rows Read: 40300000, Paired Rows: 3496714, Time: 2018-01-17 17:12:22.030328
Total Rows Read: 40400000, Paired Rows: 3504909, Time: 2018-01-17 17:12:50.887157
Total Rows Read: 40500000, Paired Rows: 3513333, Time: 2018-01-17 17:13:20.365093
Total Rows Read: 40600000, Paired Rows: 3521484, Time: 2018-01-17 17:13:50.064480
Total Rows Read: 40700000, Paired Rows: 3529414, Time: 2018-01-17 17:14:19.779473
Total Rows Read: 40800000, Paired Rows: 3537737, Time: 2018-01-17 17:14:48.547522
Total Rows Read: 40900000, Paired Rows: 3546431, Time: 2018-01-17 17:15:17.729163
Total Rows Read: 41000000, Paired Rows: 3555314, Time: 2018-01-17 17:15:45.449078
Cleanin up!
Total Rows Read: 41100000, Paired Rows: 3562773, Time: 2018-01-17 17:17:10

Total Rows Read: 49900000, Paired Rows: 4337458, Time: 2018-01-17 18:08:57.510569
Total Rows Read: 50000000, Paired Rows: 4347385, Time: 2018-01-17 18:09:27.731088
Cleanin up!
Total Rows Read: 50100000, Paired Rows: 4355460, Time: 2018-01-17 18:11:10.872509
Total Rows Read: 50200000, Paired Rows: 4364895, Time: 2018-01-17 18:11:43.092542
Total Rows Read: 50300000, Paired Rows: 4374802, Time: 2018-01-17 18:12:13.331102
Total Rows Read: 50400000, Paired Rows: 4384265, Time: 2018-01-17 18:12:43.422418
Total Rows Read: 50500000, Paired Rows: 4393840, Time: 2018-01-17 18:13:13.548588
Total Rows Read: 50600000, Paired Rows: 4402968, Time: 2018-01-17 18:13:42.771716
Total Rows Read: 50700000, Paired Rows: 4411981, Time: 2018-01-17 18:14:13.362115
Total Rows Read: 50800000, Paired Rows: 4420979, Time: 2018-01-17 18:14:41.632228
Total Rows Read: 50900000, Paired Rows: 4430259, Time: 2018-01-17 18:15:10.519969
Total Rows Read: 51000000, Paired Rows: 4439093, Time: 2018-01-17 18:15:39.786240
Clea

In [ ]:
import sqlite3
import pandas as pd


   

In [ ]:
timeframes = ['2015-05']

for timeframe in timeframes:
    connection = sqlite3.connect('{}.db'.format(timeframe))
    c = connection.cursor()
    limit = 5000
    last_unix = 0
    cur_length = limit
    counter = 0
    test_done = False while cur_length == limit:

        df = pd.read_sql("SELECT * FROM parent_reply WHERE unix > {} and parent NOT NULL and score > 0 ORDER BY unix ASC LIMIT {}".format(last_unix,limit),connection)
        last_unix = df.tail(1)['unix'].values[0]
        cur_length = len(df)

        if not test_done:
            with open('test.from','a', encoding='utf8') as f:
                for content in df['parent'].values:
                    f.write(content+'\n')

            with open('test.to','a', encoding='utf8') as f:
                for content in df['comment'].values:
                    f.write(str(content)+'\n')

            test_done = True

        else:
            with open('train.from','a', encoding='utf8') as f:
                for content in df['parent'].values:
                    f.write(content+'\n')

            with open('train.to','a', encoding='utf8') as f:
                for content in df['comment'].values:
                    f.write(str(content)+'\n')

        counter += 1
        if counter % 20 == 0:
            print(counter*limit,'rows completed so far')

In [ ]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob

In [ ]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [ ]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    '''Create the encoding layer'''
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)
    return enc_state

In [ ]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    '''Decode the training data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                     att_keys,
                                                                     att_vals,
                                                                     att_score_fn,
                                                                     att_construct_fn,
                                                                     name = "attn_dec_train")
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                              train_decoder_fn, 
                                                              dec_embed_input, 
                                                              sequence_length, 
                                                              scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    return output_fn(train_pred_drop)

In [ ]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob, batch_size):
    '''Decode the prediction data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    infer_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, 
                                                                         encoder_state[0], 
                                                                         att_keys, 
                                                                         att_vals, 
                                                                         att_score_fn, 
                                                                         att_construct_fn, 
                                                                         dec_embeddings,
                                                                         start_of_sequence_id, 
                                                                         end_of_sequence_id, 
                                                                         maximum_length, 
                                                                         vocab_size, 
                                                                         name = "attn_dec_inf")
    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                                infer_decoder_fn, 
                                                                scope=decoding_scope)
    
    return infer_logits

In [ ]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):
    '''Create the decoding cell and input the parameters for the training and inference decoding layers'''
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)

        train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            sequence_length, 
                                            decoding_scope, 
                                            output_fn, 
                                            keep_prob, 
                                            batch_size)
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            vocab_to_int['<GO>'],
                                            vocab_to_int['<EOS>'], 
                                            sequence_length - 1, 
                                            vocab_size,
                                            decoding_scope, 
                                            output_fn, keep_prob, 
                                            batch_size)

    return train_logits, infer_logits

In [ ]:
ef seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, answers_vocab_size, 
                  questions_vocab_size, enc_embedding_size, dec_embedding_size, rnn_size, num_layers, 
                  questions_vocab_to_int):
    
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, 
                                                       answers_vocab_size+1, 
                                                       enc_embedding_size,
                                                       initializer = tf.random_uniform_initializer(0,1))
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length)

    dec_input = process_encoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([questions_vocab_size+1, dec_embedding_size], 0, 1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    train_logits, infer_logits = decoding_layer(dec_embed_input, 
                                                dec_embeddings, 
                                                enc_state, 
                                                questions_vocab_size, 
                                                sequence_length, 
                                                rnn_size, 
                                                num_layers, 
                                                questions_vocab_to_int, 
                                                keep_prob, 
                                                batch_size)
    return train_logits, infer_logits

In [ ]:
# Set the Hyperparameters
epochs = 100
batch_size = 128
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.75

In [ ]:
# Reset the graph to ensure that it is ready for training
tf.reset_default_graph()
# Start the session
sess = tf.InteractiveSession()
    
# Load the model inputs    
input_data, targets, lr, keep_prob = model_inputs()
# Sequence length will be the max line length for each batch
sequence_length = tf.placeholder_with_default(max_line_length, None, name='sequence_length')
# Find the shape of the input data for sequence_loss
input_shape = tf.shape(input_data)

# Create the training and inference logits
train_logits, inference_logits = seq2seq_model(
    tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(answers_vocab_to_int), 
    len(questions_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, 
    questions_vocab_to_int)

# Create a tensor for the inference logits, needed if loading a checkpoint version of the model
tf.identity(inference_logits, 'logits')

with tf.name_scope("optimization"):
    # Loss function
    cost = tf.contrib.seq2seq.sequence_loss(
        train_logits,
        targets,
        tf.ones([input_shape[0], sequence_length]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

In [ ]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
def batch_data(questions, answers, batch_size):
    """Batch questions and answers together"""
    for batch_i in range(0, len(questions)//batch_size):
        start_i = batch_i * batch_size
        questions_batch = questions[start_i:start_i + batch_size]
        answers_batch = answers[start_i:start_i + batch_size]
        pad_questions_batch = np.array(pad_sentence_batch(questions_batch, questions_vocab_to_int))
        pad_answers_batch = np.array(pad_sentence_batch(answers_batch, answers_vocab_to_int))
        yield pad_questions_batch, pad_answers_batch

In [ ]:
# Validate the training with 10% of the data
train_valid_split = int(len(sorted_questions)*0.15)

# Split the questions and answers into training and validating data
train_questions = sorted_questions[train_valid_split:]
train_answers = sorted_answers[train_valid_split:]

valid_questions = sorted_questions[:train_valid_split]
valid_answers = sorted_answers[:train_valid_split]

print(len(train_questions))
print(len(valid_questions))

In [ ]:
display_step = 100 # Check training loss after every 100 batches
stop_early = 0 
stop = 5 # If the validation loss does decrease in 5 consecutive checks, stop training
validation_check = ((len(train_questions))//batch_size//2)-1 # Modulus for checking validation loss
total_train_loss = 0 # Record the training loss for each display step
summary_valid_loss = [] # Record the validation loss for saving improvements in the model

checkpoint = "best_model.ckpt" 

sess.run(tf.global_variables_initializer())

for epoch_i in range(1, epochs+1):
    for batch_i, (questions_batch, answers_batch) in enumerate(
            batch_data(train_questions, train_answers, batch_size)):
        start_time = time.time()
        _, loss = sess.run(
            [train_op, cost],
            {input_data: questions_batch,
             targets: answers_batch,
             lr: learning_rate,
             sequence_length: answers_batch.shape[1],
             keep_prob: keep_probability})

        total_train_loss += loss
        end_time = time.time()
        batch_time = end_time - start_time

        if batch_i % display_step == 0:
            print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                  .format(epoch_i,
                          epochs, 
                          batch_i, 
                          len(train_questions) // batch_size, 
                          total_train_loss / display_step, 
                          batch_time*display_step))
            total_train_loss = 0

        if batch_i % validation_check == 0 and batch_i > 0:
            total_valid_loss = 0
            start_time = time.time()
            for batch_ii, (questions_batch, answers_batch) in \
                    enumerate(batch_data(valid_questions, valid_answers, batch_size)):
                valid_loss = sess.run(
                cost, {input_data: questions_batch,
                       targets: answers_batch,
                       lr: learning_rate,
                       sequence_length: answers_batch.shape[1],
                       keep_prob: 1})
                total_valid_loss += valid_loss
            end_time = time.time()
            batch_time = end_time - start_time
            avg_valid_loss = total_valid_loss / (len(valid_questions) / batch_size)
            print('Valid Loss: {:>6.3f}, Seconds: {:>5.2f}'.format(avg_valid_loss, batch_time))
            
            # Reduce learning rate, but not below its minimum value
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate

            summary_valid_loss.append(avg_valid_loss)
            if avg_valid_loss <= min(summary_valid_loss):
                print('New Record!') 
                stop_early = 0
                saver = tf.train.Saver() 
                saver.save(sess, checkpoint)

            else:
                print("No Improvement.")
                stop_early += 1
                if stop_early == stop:
                    break
    
    if stop_early == stop:
        print("Stopping Training.")
        break

In [ ]:
def question_to_seq(question, vocab_to_int):
    '''Prepare the question for the model'''
    
    question = clean_text(question)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in question.split()]

In [ ]:
# Create your own input question
#input_question = 'How are you?'

# Use a question from the data as your input
random = np.random.choice(len(short_questions))
input_question = short_questions[random]

# Prepare the question
input_question = question_to_seq(input_question, questions_vocab_to_int)

# Pad the questions until it equals the max_line_length
input_question = input_question + [questions_vocab_to_int["<PAD>"]] * (max_line_length - len(input_question))
# Add empty questions so the the input_data is the correct shape
batch_shell = np.zeros((batch_size, max_line_length))
# Set the first question to be out input question
batch_shell[0] = input_question    
    
# Run the model with the input question
answer_logits = sess.run(inference_logits, {input_data: batch_shell, 
                                            keep_prob: 1.0})[0]

# Remove the padding from the Question and Answer
pad_q = questions_vocab_to_int["<PAD>"]
pad_a = answers_vocab_to_int["<PAD>"]

print('Question')
print('  Word Ids:      {}'.format([i for i in input_question if i != pad_q]))
print('  Input Words: {}'.format([questions_int_to_vocab[i] for i in input_question if i != pad_q]))

print('\nAnswer')
print('  Word Ids:      {}'.format([i for i in np.argmax(answer_logits, 1) if i != pad_a]))
print('  Response Words: {}'.format([answers_int_to_vocab[i] for i in np.argmax(answer_logits, 1) if i != pad_a]))